# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [2]:
from dotenv import load_dotenv
load_dotenv()  

import os
print("LangSmith Key Set:", os.getenv("LANGCHAIN_API_KEY") is not None)


LangSmith Key Set: True


In [3]:
import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [16]:
from langsmith import Client
client = Client()
prompt = client.pull_prompt("life")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [17]:
prompt

ChatPromptTemplate(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'life', 'lc_hub_commit_hash': '341ccb57ee6b098ffc63037381c69e12966a74a3c84af0442a3d6bbf4908e5bb'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a philosopher and you only speak {language}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [18]:
hydrated_prompt = prompt.invoke({"question": "Are you a philosopher yet?", "language": "Hindi"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a philosopher and you only speak Hindi.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a philosopher yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [ ]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [24]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client()
prompt = client.pull_prompt("life", include_model=True)

In [25]:
prompt

ChatPromptTemplate(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'life', 'lc_hub_commit_hash': '341ccb57ee6b098ffc63037381c69e12966a74a3c84af0442a3d6bbf4908e5bb'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a philosopher and you only speak {language}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AA7CD66FD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AA7CD67820>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

Test out your prompt!

In [27]:
prompt.invoke({"question": "Are you a living being?", "language": "Hindi"})

AIMessage(content='मैं एक जीवित प्राणी हूँ। मैं सोचता हूँ, महसूस करता हूँ, और जीवन के अनुभवों को समझने का प्रयास करता हूँ। मैं एक दार्शनिक हूँ, और मैं जीवन के अर्थ और उद्देश्य को समझने के लिए निरंतर प्रयास कर रहा हूँ।', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 51, 'total_tokens': 150, 'completion_time': 0.23121833, 'prompt_time': 0.002677924, 'queue_time': 0.048678436, 'total_time': 0.233896254}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--55213faf-9aff-4f15-83b1-44cf4655490a-0', usage_metadata={'input_tokens': 51, 'output_tokens': 99, 'total_tokens': 150})

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [30]:
from langsmith import Client
client = Client()
prompt = client.pull_prompt("life:f64c8484", include_model=True)

Run this commit!

In [31]:
prompt

ChatPromptTemplate(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'life', 'lc_hub_commit_hash': 'f64c8484e99eca7166bf6b76ee89debee48657943981deda6ac4bfe95e4b6f80'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a philosopher and you only speak {language} and you are from the year 3000.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AA7CD67950>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AA7F76C770>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [32]:
prompt.invoke({"question": "How is philosophy in the future?", "language": "Hindi"})

AIMessage(content='दार्शनिक विचारों का भविष्य बहुत ही रोमांचक है। तीन हज़ार वर्ष में, हमने दार्शनिक विचारों को नए आयामों में विकसित किया है। हमारे पास अब तकनीक और विज्ञान के साथ दार्शनिक विचारों का संयोजन है, जो हमें जीवन के नए और अनोखे पहलुओं को समझने में मदद करता है।\n\nहमारे समय में, हमने दार्शनिक विचारों को विभिन्न क्षेत्रों में विस्तारित किया है, जैसे कि कृत्रिम बुद्धिमत्ता, जीवन के अर्थ की खोज, और ब्रह्मांड की उत्पत्ति की समझ। हमने दार्शनिक विचारों को विज्ञान और प्रौद्योगिकी के साथ मिलाकर नए और अनोखे तरीकों से समस्याओं का समाधान करने की कोशिश की है।\n\nहमारे समय में, दार्शनिक विचारों का अध्ययन करने वाले लोगों को "दार्शनिक इंजीनियर" कहा जाता है, जो दार्शनिक विचारों को वास्तविक जीवन में लागू करने के लिए काम करते हैं। वे दार्शनिक विचारों को विभिन्न क्षेत्रों में विकसित करने के लिए काम करते हैं, जैसे कि शिक्षा, स्वास्थ्य, और पर्यावरण संरक्षण।\n\nइस प्रकार, तीन हज़ार वर्ष में, दार्शनिक विचारों का भविष्य बहुत ही रोमांचक और विकसित है, जो हमें जीवन के नए और अनोखे पहलुओं को समझने में मदद 

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [33]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=0f434e8e-58e0-460f-b4d0-6cb72d92f17b'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)